<a href="https://colab.research.google.com/github/visahan1/Spark/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    


In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 75kB/s 
     |████████████████████████████████| 204kB 18.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=4efb0c8d5b67acb75850b974120e51df4fbb79ae4b86346f9d7f7dde2ba9cad9
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [3]:
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import (StringIndexer,VectorIndexer,OneHotEncoder,VectorAssembler)
from pyspark.sql.functions import year,month
from pyspark.ml import Pipeline

In [4]:
session = SparkSession.builder.appName('churn_pred').getOrCreate()

In [6]:
data = session.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [7]:
data.head(5)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date='2013-08-13 00:38:46', Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1),
 Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date='2016-06-29 06:20:07', Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1),
 Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date='2014-04-22 12:43:12', Location='13120 Daniel Mount Angelabury, WY 30645-4695', Company='Smith Inc', Churn=1),
 Row(Names='Cynthia Norton', Age=37.0, Total_P

In [8]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [10]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [11]:
data = data.withColumn('New Date' , data['Onboard_date'].cast('Date'))

In [12]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|  New Date|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|2013-08-30|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|2013-08-13|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|2016-06-29|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-2

In [13]:
data = data.withColumn('Year',year(data['New Date']))
data = data.withColumn('Month',month(data['New Date']))
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----------+----+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|  New Date|Year|Month|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+----------+----+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|2013-08-30|2013|    8|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|2013-08-13|2013|    8|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|2016-06-29|2016|    6|
|      Phillip W

In [14]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'New Date',
 'Year',
 'Month']

In [15]:
companyIndex = StringIndexer(inputCol='Company',outputCol='Company_Indexer')
companyVec = OneHotEncoder(inputCol='Company_Indexer',outputCol='Company_Vec')


In [16]:
features = data.select('Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Company',
 'Churn',
 'Year',
 'Month')

In [26]:
assembler = VectorAssembler(inputCols=[
  'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
# 'Company_Vec',
 'Year',
 'Month'           
], outputCol='features')

In [27]:
model = LogisticRegression(featuresCol='features', labelCol='Churn')

In [28]:
#pipeline = Pipeline(stages=[companyIndex,companyVec,assembler,model])
pipeline = Pipeline(stages=[assembler,model])

In [29]:
data = data.na.drop()
train_data,test_data = data.randomSplit([0.7,0.3])

In [30]:
fit_model = pipeline.fit(train_data)

In [31]:
results = fit_model.transform(test_data)

In [33]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [35]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [37]:
roc = my_eval.evaluate(results)

In [40]:
print('ROC is {}'.format(roc))

ROC is 0.7991784037558685
